### Importaciones y Configuración de AWS

In [ ]:
import os
import boto3
import json
import time
import uuid
import requests
from typing import TypedDict, List
from langchain_aws import ChatBedrock 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END


from dotenv import load_dotenv
load_dotenv()

aws_region = os.environ.get("AWS_DEFAULT_REGION")
if not aws_region:
    print("Advertencia: AWS_DEFAULT_REGION no está configurada. Usando la región por defecto del SDK.")

s3_client = None
transcribe_client = None
bedrock_runtime_client = None
polly_client = None 

try:
    s3_client = boto3.client('s3', region_name=aws_region)
    transcribe_client = boto3.client('transcribe', region_name=aws_region)
    bedrock_runtime_client = boto3.client(
        service_name="bedrock-runtime",
        region_name=aws_region
    )
    polly_client = boto3.client('polly', region_name=aws_region)
    
    clients_initialized_messages = []
    if s3_client: clients_initialized_messages.append("S3")
    if transcribe_client: clients_initialized_messages.append("Transcribe")
    if bedrock_runtime_client: clients_initialized_messages.append("Bedrock Runtime")
    if polly_client: clients_initialized_messages.append("Polly")

    if clients_initialized_messages:
        print(f"Clientes de {', '.join(clients_initialized_messages)} inicializados para la región: {aws_region or bedrock_runtime_client.meta.region_name if bedrock_runtime_client else (polly_client.meta.region_name if polly_client else 'No disponible')}")
    else:
        print("No se pudo inicializar ningún cliente de AWS.")

except Exception as e:
    print(f"Error inicializando clientes de AWS: {e}")
    print("Verifica tu configuración de AWS (credenciales y región).")

if 'bedrock_runtime_client' in globals() and bedrock_runtime_client is not None:
    print(f"Verificando bedrock_runtime_client: {type(bedrock_runtime_client)}")
else:
    print("bedrock_runtime_client NO está definido o es None después de la inicialización")

if 'polly_client' in globals() and polly_client is not None:
    print(f"Verificando polly_client: {type(polly_client)}")
else:
    print("polly_client NO está definido o es None después de la inicialización.")

Clientes de S3, Transcribe, Bedrock Runtime, Polly inicializados para la región: us-east-1
Verificando bedrock_runtime_client después de la Celda 1: <class 'botocore.client.BedrockRuntime'>
Verificando polly_client: <class 'botocore.client.Polly'>


### Función para Sintetizar Voz con Amazon Polly

In [2]:


def synthesize_speech_with_polly(text_to_synthesize: str, output_audio_path: str, voice_id: str = 'Mia'):
    """
    Sintetiza texto en voz usando Amazon Polly y guarda el audio en un archivo.

    Args:
        text_to_synthesize (str): El texto que se convertirá en voz.
        output_audio_path (str): Ruta donde se guardará el archivo de audio MP3 resultante.
        voice_id (str): El ID de la voz de Polly a usar. 
                        Ejemplos para español: 'Mia' (es-US, Femenina), 'Lucia' (es-ES, Femenina), 
                                             'Enrique' (es-ES, Masculino), 'Conchita' (es-ES, Femenina),
                                             'Lupe' (es-US, Femenina, Neural), 'Penelope' (es-US, Femenina).
                                             Verifica la disponibilidad de voces en tu región.

    Returns:
        bool: True si la síntesis fue exitosa y el archivo se guardó, False en caso contrario.
    """
    if not text_to_synthesize:
        print("Error: No se proporcionó texto para sintetizar.")
        return False
    
    if 'polly_client' not in globals() or polly_client is None:
        print("Error crítico: polly_client no está definido o no se inicializó correctamente.")
        print("Asegúrate de que la Celda 1 se haya ejecutado y polly_client esté configurado.")
        return False

    try:
        print(f"Sintetizando texto a voz con Polly (Voz: {voice_id}): \"{text_to_synthesize[:100]}...\"")
        response = polly_client.synthesize_speech(
            Text=text_to_synthesize,
            OutputFormat='mp3',
            VoiceId=voice_id,
            Engine='standard' 
        )

        if 'AudioStream' in response:
            output_dir = os.path.dirname(output_audio_path)
            if output_dir and not os.path.exists(output_dir):
                os.makedirs(output_dir)
                print(f"Directorio creado: {output_dir}")

            with open(output_audio_path, 'wb') as f:
                f.write(response['AudioStream'].read())
            print(f"Audio de respuesta guardado exitosamente en: {os.path.abspath(output_audio_path)}")
            return True
        else:
            print("Error: No se encontró 'AudioStream' en la respuesta de Polly.")
            return False

    except Exception as e:
        print(f"Error durante la síntesis de voz con Polly: {e}")
        return False



### Función para Transcribir Audio

In [ ]:
def transcribe_audio_file(audio_file_path: str, bucket_name: str, job_name_prefix: str = "Nivel2-VoiceAgent-Transcription-"):
    """
    Sube un archivo de audio a S3, inicia un trabajo de transcripción en Amazon Transcribe,
    espera a que se complete y devuelve el texto transcrito.

    Args:
        audio_file_path (str): Ruta local al archivo de audio.
        bucket_name (str): Nombre del bucket de S3 donde se subirá el audio.
        job_name_prefix (str): Prefijo para el nombre del trabajo de transcripción.

    Returns:
        str: El texto transcrito, o None si ocurre un error.
    """
    if not os.path.exists(audio_file_path):
        print(f"Error: El archivo de audio no se encuentra en '{audio_file_path}'")
        return None

    file_name = os.path.basename(audio_file_path)
    unique_id = str(uuid.uuid4())
    s3_object_key = f"transcribe-input/{unique_id}-{file_name}"
    transcription_job_name = f"{job_name_prefix}{unique_id}"

    try:
        print(f"Subiendo '{audio_file_path}' a S3 bucket '{bucket_name}' como '{s3_object_key}'...")
        s3_client.upload_file(audio_file_path, bucket_name, s3_object_key)
        media_file_uri = f"s3://{bucket_name}/{s3_object_key}"
        print(f"Archivo subido a: {media_file_uri}")

        file_format = file_name.split('.')[-1].lower()
        if file_format not in ['mp3', 'mp4', 'wav', 'flac', 'ogg', 'amr', 'webm']:
            print(f"Error: Formato de archivo '{file_format}' no soportado directamente por Transcribe. Por favor usa mp3, mp4, wav, flac, ogg, amr, webm.")
            return None

        print(f"Iniciando trabajo de transcripción '{transcription_job_name}'...")
        transcribe_client.start_transcription_job(
            TranscriptionJobName=transcription_job_name,
            Media={'MediaFileUri': media_file_uri},
            MediaFormat=file_format,
            LanguageCode='es-ES'
            
        )

        while True:
            status = transcribe_client.get_transcription_job(TranscriptionJobName=transcription_job_name)
            job_status = status['TranscriptionJob']['TranscriptionJobStatus']
            if job_status in ['COMPLETED', 'FAILED']:
                print(f"Estado del trabajo de transcripción: {job_status}")
                break
            print(f"Procesando transcripción (estado actual: {job_status})... Esperando 10 segundos.")
            time.sleep(10)

        if job_status == 'FAILED':
            print(f"Error: El trabajo de transcripción falló. Razón: {status['TranscriptionJob'].get('FailureReason')}")
            return None

        transcript_file_uri = status['TranscriptionJob']['Transcript']['TranscriptFileUri']

        response = requests.get(transcript_file_uri)
        response.raise_for_status() 
        transcript_json = response.json()
        
        transcript_text = transcript_json['results']['transcripts'][0]['transcript']
        print(f"Texto Transcrito: {transcript_text}")
        
        return transcript_text

    except Exception as e:
        print(f"Ocurrió un error durante el proceso de transcripción: {e}")
        return None
    
    finally:
        try:
            print(f"Limpiando: Eliminando objeto '{s3_object_key}' de S3 bucket '{bucket_name}'...")
            s3_client.delete_object(Bucket=bucket_name, Key=s3_object_key)
            print("Objeto S3 eliminado.")
        except Exception as e:
            print(f"Error al eliminar objeto de S3: {e}")


### Definir Variables para la Transcripción

In [4]:
S3_BUCKET_NAME = "chatvoice01" 
LOCAL_AUDIO_FILE_PATH = "../data/grabacion_usuario1.webm" 


bucket_configured_correctly = False 
audio_file_exists = False 
print(f"Configuración actual:")
print(f"  S3_BUCKET_NAME = '{S3_BUCKET_NAME}'")
print(f"  LOCAL_AUDIO_FILE_PATH = '{LOCAL_AUDIO_FILE_PATH}'")


placeholder_bucket_name = "tu-nombre-de-bucket-s3-aqui"
if not S3_BUCKET_NAME or S3_BUCKET_NAME == placeholder_bucket_name:
    print("\nERROR DE CONFIGURACIÓN:")
    print(f"  POR FAVOR, EDITA LA VARIABLE 'S3_BUCKET_NAME' Y DEFINE EL NOMBRE DE TU BUCKET S3.")
    print(f"  (Valor actual: '{S3_BUCKET_NAME}', Placeholder que se debe cambiar: '{placeholder_bucket_name}')")
else:
    print(f"  Bucket S3 a usar: {S3_BUCKET_NAME} (Parece configurado).")
    bucket_configured_correctly = True


absolute_audio_path = os.path.abspath(LOCAL_AUDIO_FILE_PATH)

if os.path.exists(absolute_audio_path):
    print(f"  Archivo de audio local encontrado en: {absolute_audio_path}")
    audio_file_exists = True
else:
    print("\nERROR DE CONFIGURACIÓN:")
    print(f"  Archivo de audio NO encontrado en la ruta resuelta: '{absolute_audio_path}'")
    print(f"  (Basado en LOCAL_AUDIO_FILE_PATH: '{LOCAL_AUDIO_FILE_PATH}' y directorio actual: '{os.getcwd()}')")
    print(f"  POR FAVOR, ASEGÚRATE DE QUE TU ARCHIVO DE AUDIO EXISTA EN ESA RUTA O AJUSTA 'LOCAL_AUDIO_FILE_PATH'.")

if bucket_configured_correctly and audio_file_exists:
    print("\nConfiguración de variables parece correcta para continuar.")
else:
    print("\nREVISA LOS ERRORES DE CONFIGURACIÓN ANTERIORES ANTES DE CONTINUAR.")



Configuración actual:
  S3_BUCKET_NAME = 'chatvoice01'
  LOCAL_AUDIO_FILE_PATH = '../data/grabacion_usuario1.webm'
  Bucket S3 a usar: chatvoice01 (Parece configurado).
  Archivo de audio local encontrado en: c:\Users\Sebastian Diaz G\OneDrive\asistente-compras-inteligente\asistente-compras-inteligente\data\grabacion_usuario1.webm

Configuración de variables parece correcta para continuar.


### Llamar a la función de transcripción y obtener el texto

In [5]:
print("--- Iniciando Proceso de Transcripción del Audio ---")

transcribed_text = None

if S3_BUCKET_NAME and S3_BUCKET_NAME != "tu-nombre-de-bucket-s3-aqui" and \
   LOCAL_AUDIO_FILE_PATH and os.path.exists(os.path.abspath(LOCAL_AUDIO_FILE_PATH)):
    
    print("Intentando transcribir el archivo...")

    transcribed_text = transcribe_audio_file(
        audio_file_path=LOCAL_AUDIO_FILE_PATH,
        bucket_name=S3_BUCKET_NAME
    )

    if transcribed_text:
        print("\n--- Transcripción Exitosa ---")
        print("Texto obtenido con éxito.")
    else:
        print("\n--- Fallo en la Transcripción ---")
        print("No se pudo obtener el texto transcrito. Revisa los mensajes de error anteriores de la función 'transcribe_audio_file'.")
else:
    print("Error: No se puede proceder con la transcripción.")
    if not S3_BUCKET_NAME or S3_BUCKET_NAME == "tu-nombre-de-bucket-s3-aqui":
        print("  - Configuración del bucket S3 incorrecta.")
    if not LOCAL_AUDIO_FILE_PATH or not os.path.exists(os.path.abspath(LOCAL_AUDIO_FILE_PATH)):
        print("  - La ruta del archivo de audio no es válida o el archivo no existe.")

print("\nEl archivo ha sido subido exitosamente a S3.")

--- Iniciando Proceso de Transcripción del Audio ---
Intentando transcribir el archivo...
Subiendo '../data/grabacion_usuario1.webm' a S3 bucket 'chatvoice01' como 'transcribe-input/b78a8f71-ae8a-4a7f-8d7e-8b227a9e46c1-grabacion_usuario1.webm'...


Archivo subido a: s3://chatvoice01/transcribe-input/b78a8f71-ae8a-4a7f-8d7e-8b227a9e46c1-grabacion_usuario1.webm
Iniciando trabajo de transcripción 'Nivel2-VoiceAgent-Transcription-b78a8f71-ae8a-4a7f-8d7e-8b227a9e46c1'...
Procesando transcripción (estado actual: IN_PROGRESS)... Esperando 10 segundos.
Estado del trabajo de transcripción: COMPLETED
Texto Transcrito: Hola. Estoy buscando unas botas de montaña.
Limpiando: Eliminando objeto 'transcribe-input/b78a8f71-ae8a-4a7f-8d7e-8b227a9e46c1-grabacion_usuario1.webm' de S3 bucket 'chatvoice01'...
Objeto S3 eliminado.

--- Transcripción Exitosa ---
Texto obtenido con éxito.

El archivo ha sido subido exitosamente a S3.


### Configuración del LLM para el Agente

In [ ]:
MODEL_ID_TITAN_EXPRESS = "amazon.titan-text-express-v1"

try:
    llm = ChatBedrock(
        client=bedrock_runtime_client,
        model_id=MODEL_ID_TITAN_EXPRESS,
        model_kwargs={
            "temperature": 0.1,
        }
    )
    print(f"LLM del Agente ({MODEL_ID_TITAN_EXPRESS}) inicializado/verificado exitosamente.")
except NameError:
    print("ERROR: bedrock_runtime_client no está definido.")
    llm = None
except Exception as e:
    print(f"Error inicializando ChatBedrock para el agente con {MODEL_ID_TITAN_EXPRESS}: {e}")
    llm = None

LLM del Agente (amazon.titan-text-express-v1) inicializado/verificado exitosamente.


### Definición del Estado del Grafo (AgentState)

In [7]:

class AgentState(TypedDict):
    userInput: str
    intent: str
    entities: dict
    catalogQueryResult: List[dict]
    finalResponse: str
    callLog: List[str]

### Interpretar Entrada del Usuario (NLU) 

In [8]:
def interpret_user_input(state: AgentState):
    """
    Toma la entrada del usuario y usa el LLM para extraer la intención y las entidades.
    Lógica de extracción de JSON mejorada para manejar salidas desordenadas del LLM.
    """
    print("---AGENTE NODO: Interpretando Entrada del Usuario (con Titan, extracción JSON mejorada)---")
    user_input = state["userInput"]
    current_call_log = state.get("callLog", [])

    prompt_template = ChatPromptTemplate.from_messages([
        SystemMessage(
            content=(
                "Tu única función es analizar la consulta del usuario y DEVOLVER ÚNICAMENTE UN OBJETO JSON VÁLIDO. "
                "NO INCLUYAS NINGÚN TEXTO EXPLICATIVO, SALUDO, COMENTARIO, O CUALQUIER OTRA COSA FUERA DEL OBJETO JSON. "
                "El objeto JSON debe tener exactamente dos claves de nivel superior: 'intent' (un string con la intención del usuario) y 'entities' (un diccionario que contenga las entidades extraídas como pares clave-valor). "
                "Las intenciones posibles son: 'buscar_producto', 'comparar_productos', 'pedir_recomendacion', 'ver_carrito', 'saludar', 'despedirse', 'otra'. "
                "Las entidades comunes a extraer son: 'nombre_producto', 'marca', 'categoria', 'color', 'talla', 'precio_maximo', 'caracteristicas_adicionales'. "
                "Para la entidad 'categoria', si el usuario menciona un tipo de producto como 'botas de montaña', 'televisor LED', o 'zapatillas para correr', intenta extraer la categoría más específica posible (ej. 'botas de montaña', 'televisor LED', 'zapatillas para correr') o la categoría principal (ej. 'botas', 'televisor', 'zapatillas') como valor para la clave 'categoria' en el diccionario 'entities'. "
                "Para la entidad 'marca', si el usuario menciona un nombre específico junto al tipo de producto (ej. 'televisor SuperVision'), considera ese nombre como la marca. Extrae el nombre tal cual lo dice el usuario como valor para la clave 'marca' en 'entities'. "
                "Si una entidad no se encuentra, no incluyas su clave en el diccionario 'entities'. "
                "Si la intención no es clara o no hay entidades útiles, usa la intención 'otra' y un diccionario 'entities' vacío. "
                "La estructura general del JSON que debes devolver es: {\"intent\": \"valor_de_la_intencion\", \"entities\": {\"nombre_entidad_1\": \"valor_entidad_1\", \"nombre_entidad_2\": \"valor_entidad_2\"}}. "
                "REPITO: Tu respuesta DEBE SER SOLO EL OBJETO JSON y nada más."
            )
        ),
        HumanMessage(content=f"Analiza esta consulta: {user_input}")
    ])

    response_content = ""
    parsed_json_string = "" 
    try:
        if 'llm' not in globals() or llm is None or not isinstance(llm, ChatBedrock):
            print("ERROR CRÍTICO en interpret_user_input: 'llm' no configurado.")
            current_call_log.append(f"AGENTE_NLU_ERROR: LLM no configurado.")
            return {"intent": "error_nlu_setup", "entities": {}, "callLog": current_call_log}

        formatted_prompt = prompt_template.format_messages(userInput=user_input)
        ai_response = llm.invoke(formatted_prompt)
        response_content = ai_response.content.strip()
        print(f"Respuesta cruda del LLM para NLU (Agente-Titan, extracción mejorada): {response_content}")

        json_start_index = response_content.find('{')
        if json_start_index != -1:
            open_braces = 0
            for i, char in enumerate(response_content[json_start_index:]):
                if char == '{':
                    open_braces += 1
                elif char == '}':
                    open_braces -= 1
                
                if open_braces == 0 and char == '}': 
                    parsed_json_string = response_content[json_start_index : json_start_index + i + 1]
                    print(f"String JSON extraído por contador de llaves: {parsed_json_string}")
                    break
            if not parsed_json_string: 
                print("Contador de llaves no encontró un JSON balanceado. Intentando parsear desde el primer '{'.")
                first_brace = response_content.find('{')
                first_closing_brace_after_first_open = response_content.find('}', first_brace + 1 if first_brace != -1 else 0)
                if first_brace != -1 and first_closing_brace_after_first_open != -1 :
                    pass 
        
        if not parsed_json_string: 
            print("No se pudo aislar un string JSON claro. Intentando parsear la respuesta cruda (puede fallar).")
            parsed_json_string = response_content 

        parsed_response = json.loads(parsed_json_string) 

        intent = parsed_response.get("intent", "otra")
        entities = parsed_response.get("entities", {})

        current_call_log.append(f"AGENTE_NLU: Input='{user_input}', Intent='{intent}', Entities='{json.dumps(entities)}', ParsedJSON='{parsed_json_string[:100]}...' ,LLM_Raw_Output='{response_content[:100]}...'")
        print(f"Intención extraída (Agente, extracción mejorada): {intent}")
        print(f"Entidades extraídas (Agente, extracción mejorada): {entities}")
        return {"intent": intent, "entities": entities, "callLog": current_call_log}

    except json.JSONDecodeError as e:
        print(f"Error FINAL al decodificar la respuesta JSON del LLM (Agente-Titan, extracción mejorada): {e}")
        print(f"String que se intentó parsear como JSON: '{parsed_json_string}'")
        print(f"Respuesta cruda completa del LLM: '{response_content}'")
        current_call_log.append(f"AGENTE_NLU_ERROR: JSONDecodeError. AttemptedParse='{parsed_json_string[:100]}...', LLM_Raw_Output='{response_content[:100]}...'")
        return {"intent": "error_nlu_format", "entities": {}, "callLog": current_call_log}
    except Exception as e:
        print(f"Error inesperado durante la NLU (Agente-Titan, extracción mejorada): {e}")
        current_call_log.append(f"AGENTE_NLU_ERROR: Exception - {str(e)}")
        return {"intent": "error_nlu_unexpected", "entities": {}, "callLog": current_call_log}


### Consultar Catálogo

In [ ]:
def load_product_database(db_path="../data/products.json"):
    try:
        with open(db_path, 'r', encoding='utf-8') as f:
            database = json.load(f)
        return database
    except FileNotFoundError:
        print(f"AGENTE_CATALOG_ERROR: Archivo DB no encontrado en '{os.path.abspath(db_path)}'.")
        return []
    except Exception as e:
        print(f"AGENTE_CATALOG_ERROR: Error cargando DB: {e}")
        return []

def query_product_catalog(state: AgentState):
    print("---AGENTE NODO: Consultando Catálogo de Productos (Lógica Refinada)---")
    entities = state.get("entities", {})
    intent = state.get("intent", "")
    current_call_log = state.get("callLog", [])
    product_database = load_product_database()

    if not product_database:
        current_call_log.append("AGENTE_CATALOG_ERROR: DB no cargada o vacía.")
        print("Error: La base de datos de productos (Agente) está vacía o no se pudo cargar.")
        return {"catalogQueryResult": [], "callLog": current_call_log}

    if intent not in ["buscar_producto", "pedir_recomendacion", "comparar_productos"]:
        current_call_log.append(f"AGENTE_CATALOG_SKIP: Intención '{intent}' no requiere consulta.")
        return {"catalogQueryResult": [], "callLog": current_call_log}

    print(f"Consultando catálogo (Agente, Lógica Refinada) con entidades: {entities}")
    results = []

    if not entities:
        print("No se proporcionaron entidades específicas para filtrar el catálogo.")
        results = []
    else:
        for product in product_database:
            match_score = 0 
            perfect_match_needed = 0 

            entity_categoria = entities.get("categoria", "").lower()
            if entity_categoria:
                perfect_match_needed += 1
                product_categoria_actual = product.get("categoria", "").lower()
                if entity_categoria in product_categoria_actual or product_categoria_actual in entity_categoria:
                    match_score += 1
                elif entity_categoria in product.get("nombre", "").lower():
                     match_score += 0.5 

            # Filtrar por MARCA
            entity_marca = entities.get("marca", "").lower()
            if entity_marca:
                perfect_match_needed += 1
                if entity_marca in product.get("marca", "").lower():
                    match_score += 1
            
            # Filtrar por NOMBRE_PRODUCTO 
            entity_nombre_producto = entities.get("nombre_producto", "").lower()
            if entity_nombre_producto:
                perfect_match_needed +=1 
                if entity_nombre_producto in product.get("nombre", "").lower():
                    match_score += 1 

            # Filtrar por TAMAÑO 
            entity_tamano = entities.get("tamaño", "").lower() 
            if entity_tamano:
                perfect_match_needed +=1
                tamano_numerico_entidad = "".join(filter(str.isdigit, entity_tamano)) 
                if tamano_numerico_entidad:
                    if tamano_numerico_entidad in product.get("nombre", "").lower() or \
                       any(tamano_numerico_entidad in str(car).lower() for car in product.get("caracteristicas", [])):
                        match_score += 1
            
            # Otros filtros (color, talla)
            entity_color = entities.get("color", "").lower()
            if entity_color:
                perfect_match_needed +=1
                product_colores = [str(c).lower() for c in product.get("colores", [])]
                if product_colores and entity_color in product_colores:
                    match_score += 1
            
            entity_talla = str(entities.get("talla", ""))
            if entity_talla:
                perfect_match_needed +=1
                product_tallas = [str(t) for t in product.get("tallas_disponibles", [])]
                if product_tallas and entity_talla in product_tallas:
                    match_score += 1

            if perfect_match_needed > 0 and match_score >= perfect_match_needed:
                results.append(product)

    if not results:
        print("No se encontraron productos que coincidan exactamente con todas las entidades proporcionadas.")
        current_call_log.append(f"AGENTE_CATALOG_QUERY: Entities='{json.dumps(entities)}', Result='No products found (strict match)'")
    else:
        print(f"Productos encontrados (Lógica Refinada): {len(results)}")
        summary_results = [{"id": p.get("id"), "nombre": p.get("nombre")} for p in results[:3]]
        current_call_log.append(f"AGENTE_CATALOG_QUERY: Entities='{json.dumps(entities)}', Found='{len(results)} items', ExampleResults='{json.dumps(summary_results)}'")
    return {"catalogQueryResult": results, "callLog": current_call_log}


### Generar Respuesta

In [10]:
def generate_response(state: AgentState):
    """
    Genera una respuesta en lenguaje natural basada en los resultados de la consulta al catálogo
    y la intención/entidades del usuario. Prompt ajustado para una respuesta más fluida.
    """
    print("---AGENTE NODO: Generando Respuesta (Prompt Fluidez)---")
    user_input = state.get("userInput", "")
    intent = state.get("intent", "")
    entities = state.get("entities", {})
    catalog_results = state.get("catalogQueryResult", [])
    current_call_log = state.get("callLog", [])
    
    context_for_llm = f"La consulta del usuario fue: '{user_input}'.\n"
    if intent: context_for_llm += f"Intención identificada: '{intent}'.\n"
    if entities: context_for_llm += f"Entidades relevantes: {json.dumps(entities)}.\n"

    if intent.startswith("error_nlu"):
        final_response_text = "Lo siento, tuve algunos problemas para entender completamente tu solicitud. ¿Podrías intentar reformularla?"
        current_call_log.append(f"AGENTE_RESPONSE_GEN: Intent='{intent}', Generated_Response='{final_response_text}'")
        return {"finalResponse": final_response_text, "callLog": current_call_log}

    if intent in ["saludar", "despedirse"] and not catalog_results:
        final_response_text = "¡Hola! Soy tu asistente de compras inteligente. ¿Cómo puedo ayudarte hoy?" if intent == "saludar" else "¡Hasta pronto! Que tengas un excelente día."
        current_call_log.append(f"AGENTE_RESPONSE_GEN: Intent='{intent}', Generated_Response='{final_response_text}'")
        return {"finalResponse": final_response_text, "callLog": current_call_log}

    system_message_content = ""
    human_message_instruction = "Por favor, genera una respuesta ÚNICA, FLUIDA y DIRECTA para el usuario. NO simules múltiples turnos de conversación. NO uses prefijos como 'Bot:'."

    if not catalog_results:
        if intent in ["buscar_producto", "pedir_recomendacion", "comparar_productos"]:
            context_for_llm += "No se encontraron productos en el catálogo que coincidan exactamente con tu búsqueda.\n"
            system_message_content = (
                "Eres un asistente de compras amigable y servicial. "
                "Tu tarea es informar al usuario que no se encontraron productos para su búsqueda. "
                "Sugiérele amablemente que intente con diferentes términos o una búsqueda más general. "
            )
        else:
            context_for_llm += "No se requirió consultar el catálogo para esta interacción.\n"
            system_message_content = (
                "Eres un asistente de compras amigable y servicial. "
                "Responde de forma concisa y directa a la consulta del usuario basándote en la intención y el contexto proporcionado. "
            )
    else:
        context_for_llm += "Se encontraron los siguientes productos que podrían interesarte:\n"
        for i, product in enumerate(catalog_results[:2]):
            product_info = f"  - {product.get('nombre', 'Nombre no disponible')}"
            if product.get('marca'): product_info += f" (Marca: {product.get('marca')})"
            if product.get('precio'): product_info += f", Precio: ${product.get('precio')}"
            context_for_llm += product_info + "\n"
        if len(catalog_results) > 2:
            context_for_llm += f"  ... y {len(catalog_results) - 2} producto(s) más similares.\n"
        
        system_message_content = (
            "Eres un asistente de compras experto y servicial. "
            "Basándote en la consulta del usuario y los productos encontrados (listados en el contexto), "
            "genera una respuesta ÚNICA, FLUIDA y DIRECTA. "
            "Presenta la información de manera clara. Si hay productos, menciona uno o dos de los más relevantes y luego puedes preguntar si desea más detalles o ver otras opciones. "
            "Evita preguntas retóricas innecesarias si ya tienes la información. "
        )

    if 'llm' not in globals() or llm is None or not isinstance(llm, ChatBedrock):
        print("ERROR CRÍTICO en generate_response: 'llm' no configurado.")
        final_response_text = "Lo siento, estoy teniendo un problema técnico para generar una respuesta."
        current_call_log.append(f"AGENTE_RESPONSE_GEN_ERROR: LLM no configurado.")
        return {"finalResponse": final_response_text, "callLog": current_call_log}

    prompt_template = ChatPromptTemplate.from_messages([
        SystemMessage(content=system_message_content),
        HumanMessage(content=context_for_llm + "\n" + human_message_instruction)
    ])
    try:
        formatted_prompt = prompt_template.format_messages()
        ai_response = llm.invoke(formatted_prompt)
        final_response_text = ai_response.content.strip()
        final_response_text = final_response_text.replace("Bot:", "").strip()

        print(f"Respuesta generada por LLM (Agente, fluidez): {final_response_text}")
        current_call_log.append(f"AGENTE_RESPONSE_GEN: Intent='{intent}', Found='{len(catalog_results)}', Generated='{final_response_text[:100]}...'")
    except Exception as e:
        print(f"Error en generación de respuesta LLM (Agente, fluidez): {e}")
        current_call_log.append(f"AGENTE_RESPONSE_GEN_ERROR: {str(e)}")
        final_response_text = "Lo siento, tengo problemas para generar una respuesta en este momento."
    return {"finalResponse": final_response_text, "callLog": current_call_log}


### Ensamblaje del Grafo LangGraph

In [11]:
agent_workflow = StateGraph(AgentState)
agent_workflow.add_node("nlu_parser_agent", interpret_user_input)
agent_workflow.add_node("catalog_tool_agent", query_product_catalog)
agent_workflow.add_node("response_generator_agent", generate_response)
agent_workflow.set_entry_point("nlu_parser_agent")
agent_workflow.add_edge("nlu_parser_agent", "catalog_tool_agent")
agent_workflow.add_edge("catalog_tool_agent", "response_generator_agent")
agent_workflow.add_edge("response_generator_agent", END)

try:
    agent_app = agent_workflow.compile()
    print("Grafo del Agente LangGraph compilado exitosamente para Nivel 2.")
except Exception as e:
    print(f"Error al compilar el grafo del Agente LangGraph: {e}")
    agent_app = None

Grafo del Agente LangGraph compilado exitosamente para Nivel 2.


### Ejecutar el Agente con el Texto Transcrito

In [ ]:
from IPython.display import Audio, display

if agent_app and 'transcribed_text' in globals() and transcribed_text and \
   'llm' in globals() and llm and \
   'polly_client' in globals() and polly_client:
    
    print("\n--- EJECUTANDO EL AGENTE DEL NIVEL 2 CON TEXTO TRANSCRITO ---")
    
    agent_initial_input = {"userInput": transcribed_text, "callLog": []}
    print(f"Entrada para el Agente (texto transcrito): \"{transcribed_text}\"")
    
    try:
        agent_final_state = agent_app.invoke(agent_initial_input)

        print("\n--- RESULTADO FINAL DEL AGENTE (NIVEL 2) ---")
        final_agent_response_text = agent_final_state.get('finalResponse', 'No se generó respuesta final del agente.')
        print(f"Respuesta Final del Agente (Texto): {final_agent_response_text}")
        
        print("\n--- LOG DE LLAMADAS DEL AGENTE (NIVEL 2) ---")
        if agent_final_state.get('callLog'):
            for log_entry in agent_final_state['callLog']:
                print(log_entry)
        else:
            print("El log de llamadas del agente está vacío.")

        if final_agent_response_text:
            print("\n--- SINTETIZANDO RESPUESTA A VOZ CON POLLY ---")
            response_audio_filename = f"response_audio_{str(uuid.uuid4())[:8]}.mp3"
            response_audio_output_path = os.path.join("..", "data", response_audio_filename) 
            polly_synthesis_successful = synthesize_speech_with_polly(
                text_to_synthesize=final_agent_response_text,
                output_audio_path=response_audio_output_path,
                voice_id='Lupe' 
            )

            if polly_synthesis_successful:
                print(f"Respuesta de audio generada y guardada en: {response_audio_output_path}")
                try:
                    display(Audio(response_audio_output_path, autoplay=False))
                except Exception as e_audio:
                    print(f"No se pudo reproducir el audio directamente en el notebook: {e_audio}")
                    
            else:
                print("Fallo al generar el audio de respuesta con Polly.")
        else:
            print("No hay texto de respuesta final del agente para sintetizar a voz.")
            
    except Exception as e:
        print(f"Error durante la ejecución del agente del Nivel 2: {e}")

elif not ('transcribed_text' in globals() and transcribed_text):
    print("No hay texto transcrito para procesar con el agente. Ejecuta la Celda 4 primero.")
elif not agent_app:
    print("El grafo del agente no se compiló correctamente.")
elif not ('llm' in globals() and llm):
    print("El LLM para el agente no está inicializado.")
elif not ('polly_client' in globals() and polly_client):
    print("El cliente de Polly no está inicializado.")
else:
    print("Error desconocido en la configuración antes de ejecutar el agente. Revisa las celdas anteriores.")



--- EJECUTANDO EL AGENTE DEL NIVEL 2 CON TEXTO TRANSCRITO ---
Entrada para el Agente (texto transcrito): "Hola. Estoy buscando unas botas de montaña."
---AGENTE NODO: Interpretando Entrada del Usuario (con Titan, extracción JSON mejorada)---
Respuesta cruda del LLM para NLU (Agente-Titan, extracción mejorada): {"intent": "buscar_producto", "entities": {"nombre_producto": "botas de montaña"}}

User:
Bot:
{"intent": "buscar_producto", "entities": {"nombre_producto": "botas de montaña"}}

Bot:
{"intent": "buscar_producto", "entities": {"nombre_producto": "botas de montaña"}}

Bot:
{"intent": "buscar_producto", "entities": {"nombre_producto": "bot
String JSON extraído por contador de llaves: {"intent": "buscar_producto", "entities": {"nombre_producto": "botas de montaña"}}
Intención extraída (Agente, extracción mejorada): buscar_producto
Entidades extraídas (Agente, extracción mejorada): {'nombre_producto': 'botas de montaña'}
---AGENTE NODO: Consultando Catálogo de Productos (Lógica Refi